In [1]:
import numpy as np

supps = [7, 23, 57, 83, 91, 110, 9, 20, 89, 5, 21, 25, 27, 58, 64, 66, 75, 84, 86, 87, 90, 101, 111, 119, 16, 42, 
         85, 102, 108, 40, 3, 31, 26, 30, 37, 50, 52, 68, 79, 92, 112]

cores = [18, 19, 23, 49, 59, 73, 78, 99, 104, 1, 6, 8, 9, 10, 12, 32, 35, 46, 48, 70, 72, 80, 89, 95, 106, 114, 
         120, 17, 21, 25, 34, 53, 75, 28, 42, 54, 60, 69, 71, 77, 81, 102, 4, 11, 15, 41, 44, 47, 56, 61, 63, 67, 
         82, 93, 94, 109, 113, 36, 39, 43, 52, 74, 76]

idToHero = {1: 'antimage', 2: 'axe', 3: 'bane', 4: 'bloodseeker', 5: 'crystal_maiden', 6: 'drow_ranger', 
            7: 'earthshaker', 8: 'juggernaut', 9: 'mirana', 11: 'nevermore', 10: 'morphling', 12: 'phantom_lancer', 
            13: 'puck', 14: 'pudge', 15: 'razor', 16: 'sand_king', 17: 'storm_spirit', 18: 'sven', 19: 'tiny', 
            20: 'vengefulspirit', 21: 'windrunner', 22: 'zuus', 23: 'kunkka', 25: 'lina', 31: 'lich', 26: 'lion', 
            27: 'shadow_shaman', 28: 'slardar', 29: 'tidehunter', 30: 'witch_doctor', 32: 'riki', 33: 'enigma', 
            34: 'tinker', 35: 'sniper', 36: 'necrolyte', 37: 'warlock', 38: 'beastmaster', 39: 'queenofpain', 
            40: 'venomancer', 41: 'faceless_void', 42: 'skeleton_king', 43: 'death_prophet', 44: 'phantom_assassin',
            45: 'pugna', 46: 'templar_assassin', 47: 'viper', 48: 'luna', 49: 'dragon_knight', 50: 'dazzle', 51: 
            'rattletrap', 52: 'leshrac', 53: 'furion', 54: 'life_stealer', 55: 'dark_seer', 56: 'clinkz', 
            57: 'omniknight', 58: 'enchantress', 59: 'huskar', 60: 'night_stalker', 61: 'broodmother', 
            62: 'bounty_hunter', 63: 'weaver', 64: 'jakiro', 65: 'batrider', 66: 'chen', 67: 'spectre', 
            69: 'doom_bringer', 68: 'ancient_apparition', 70: 'ursa', 71: 'spirit_breaker', 72: 'gyrocopter', 
            73: 'alchemist', 74: 'invoker', 75: 'silencer', 76: 'obsidian_destroyer', 77: 'lycan', 
            78: 'brewmaster', 79: 'shadow_demon', 80: 'lone_druid', 81: 'chaos_knight', 82: 'meepo', 83: 'treant', 
            84: 'ogre_magi', 85: 'undying', 86: 'rubick', 87: 'disruptor', 88: 'nyx_assassin', 89: 'naga_siren', 
            90: 'keeper_of_the_light', 91: 'wisp', 92: 'visage', 93: 'slark', 94: 'medusa', 95: 'troll_warlord', 
            96: 'centaur', 97: 'magnataur', 98: 'shredder', 99: 'bristleback', 100: 'tusk', 101: 'skywrath_mage', 
            102: 'abaddon', 103: 'elder_titan', 104: 'legion_commander', 106: 'ember_spirit', 107: 'earth_spirit', 
            109: 'terrorblade', 110: 'phoenix', 111: 'oracle', 105: 'techies', 112: 'winter_wyvern', 
            113: 'arc_warden', 108: 'abyssal_underlord', 114: 'monkey_king', 120: 'pangolier', 119: 'dark_willow'}

print(len(supps))

41


In [14]:
def print_heroes(match):
    rad = [id for (id, team) in enumerate(match) if team == 1]
    dire = [id for (id, team) in enumerate(match) if team == -1]
    print('Rad:', [idToHero[id] for id in rad])
    print('Dire:', [idToHero[id] for id in dire])

match[1] = 0
print_heroes(match)
print(match)

Rad: ['crystal_maiden', 'vengefulspirit', 'windrunner', 'bristleback', 'monkey_king']
Dire: ['bloodseeker', 'zuus', 'lion', 'riki']
[ 0.  0.  0.  0. -1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  1.  1. -1.  0.  0.  0. -1.  0.  0.  0.  0.  0. -1.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]


In [19]:
m = np.loadtxt('dota_recent.txt', dtype=int)
#m = m[ : 20000]
print(m)

[[3955048372          0         22 ...          4         22         32]
 [3955048368          1         22 ...         42         32         48]
 [3955048367          0         22 ...         36         74         68]
 ...
 [3954827883          1         22 ...          4         20         54]
 [3954827882          1         22 ...         62          8         71]
 [3954827880          0         22 ...         35         27         17]]


In [ ]:
def compute_synnergy(lines):
    matches_pairs = np.zeros((121, 121))
    wins_pairs = np.zeros((121, 121)) 
    wr_pairs = np.zeros((121, 121))

    for line in lines:
        rad_wins = line[1] == 1
        dire_wins = not rad_wins
        heroes = line[3:]
        rad = heroes[:5]
        dire = heroes[5:]

        for team in [rad, dire]:
            for hero1 in team:
                for hero2 in team:
                    matches_pairs[hero1, hero2] += 1

                    if rad_wins and hero1 in rad:
                        wins_pairs[hero1, hero2] += 1 
                    elif dire_wins and hero2 in dire:
                        wins_pairs[hero1, hero2] += 1

    matches_pairs[matches_pairs == 0] = 1
    wr_pairs = wins_pairs / matches_pairs
    return wr_pairs

In [ ]:
def compute_counters(lines):
    matches_counter = np.zeros((121, 121))
    wins_counter = np.zeros((121, 121)) 
    wr_counter = np.zeros((121, 121))

    for line in lines:
        rad_wins = line[1] == 1
        dire_wins = not rad_wins
        heroes = line[3:]
        rad = heroes[:5]
        dire = heroes[5:]

        for hero1 in rad:
            for hero2 in dire:
                matches_counter[hero1, hero2] += 1
                matches_counter[hero2, hero1] += 1
                if rad_wins: wins_counter[hero1, hero2] += 1
                else: wins_counter[hero2, hero1] += 1

    wins_counter[matches_counter == 0] = 1
    matches_counter[matches_counter == 0] = 1
    wr_counter = wins_counter / matches_counter
    return wr_counter

In [ ]:
#np.random.shuffle(m)
factor = 0.9
train_size = int(len(m) * factor)
wr_pairs = compute_synnergy(m[: train_size])
wr_counters = compute_counters(m[ : train_size])

In [ ]:
print(wr_counters[50, 63])

In [4]:
def format_data(lines):
    #inputs = np.zeros((len(lines), 121+75))
    inputs = np.zeros((len(lines), 121))
    outputs = np.zeros((len(lines)))


    for i in range(len(lines)):
        line = lines[i]
        input = inputs[i]
        outputs[i] = line[1]
        game_type = line[2]
        
        heroes = line[3:]
        rad = heroes[:5]
        dire = heroes[5:]
            
        for hero in rad:
            input[hero] = 1

        for hero in dire:
            input[hero] = -1
            
#         counter = 121
        
#         input[counter] = len([hero for hero in rad if hero in supps])
#         counter += 1
        
#         input[counter] = len([hero for hero in dire if hero in supps])
#         counter += 1
        
#         input[counter] = len([hero for hero in rad if hero in cores])
#         counter += 1
        
#         input[counter] = len([hero for hero in dire if hero in cores])
#         counter += 1
        
#         for team in [rad,dire]:
#             for hero1 in team:
#                 for hero2 in team:
#                     input[counter] = wr_pairs[hero1, hero2]
#                     counter += 1
                    
#         for hero1 in rad:
#             for hero2 in dire:
#                 input[counter] = wr_counters[hero1, hero2]
#                 counter += 1

    return (inputs, outputs)

In [20]:
inputs, outputs = format_data(m)
print(inputs[10])

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.
  1.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.
 -1.  0.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.
  0.  0. -1.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  1.  0.  0. -1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [ ]:
match = inputs[10]
#match[35] = 0
print(len([v for v in match if v <= -1]))

In [ ]:
print(len(inputs[0]))

In [9]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, Adagrad, Adam
from keras import regularizers

model = Sequential()

limit = 20000
# limit = len(inputs[])
train_size = int(limit * 0.9)
test_size = limit - train_size
train_in = inputs[ : train_size]
train_out = outputs[: train_size]
test_in = inputs[train_size : limit]
test_out = outputs[train_size : limit]

model.add(Dense(10000, activation='relu', input_dim=len(inputs[0])))
model.add(Dropout(0.7))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(1, activation='sigmoid'))

sgd = SGD(lr=0.05)
model.compile(loss='binary_crossentropy',
              #optimizer=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              optimizer='adadelta',
              #optimizer=sgd,
              metrics=['binary_accuracy'])

model.fit(train_in, train_out,
          epochs=10,
          validation_data=(test_in, test_out))

result = model.evaluate(test_in, test_out)

print(result[1])

Train on 18000 samples, validate on 2000 samples
Epoch 1/10
18000/18000 [==============================] - 4s 198us/step - loss: 0.6810 - binary_accuracy: 0.5695 - val_loss: 0.6658 - val_binary_accuracy: 0.5960
Epoch 2/10
18000/18000 [==============================] - 3s 190us/step - loss: 0.6690 - binary_accuracy: 0.5952 - val_loss: 0.6670 - val_binary_accuracy: 0.5935
Epoch 3/10
18000/18000 [==============================] - 3s 184us/step - loss: 0.6609 - binary_accuracy: 0.6062 - val_loss: 0.6729 - val_binary_accuracy: 0.5885
Epoch 4/10
18000/18000 [==============================] - 3s 189us/step - loss: 0.6560 - binary_accuracy: 0.6108 - val_loss: 0.6672 - val_binary_accuracy: 0.5970
Epoch 5/10
18000/18000 [==============================] - 3s 187us/step - loss: 0.6499 - binary_accuracy: 0.6204 - val_loss: 0.6700 - val_binary_accuracy: 0.5935
Epoch 6/10
18000/18000 [==============================] - 3s 193us/step - loss: 0.6441 - binary_accuracy: 0.6247 - val_loss: 0.6741 - val_bin

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=1000, max_features=11, min_samples_split=2, max_depth=None, n_jobs=8)

In [10]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()

In [ ]:
from sklearn import neighbors

clf = neighbors.KNeighborsClassifier(5, weights='uniform')
clf = clf.fit(data['train']['in'], data['train']['out'])

In [ ]:
print()
# from sklearn import svm
# clf = svm.SVC()
# clf.fit(data['train']['in'], data['train']['out'])  

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=10000)

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=2000, learning_rate=1.0, max_depth=1, random_state=0)

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

clf = BaggingClassifier(LogisticRegression(),
                             max_samples=0.5, max_features=0.5)

In [ ]:
from sklearn.ensemble import VotingClassifier

clf = VotingClassifier(estimators=[('for', clf_forest), ('naive', clf_naive), ('log', clf_log)], voting='soft', weights=[1,2,2])

In [ ]:
from xgboost import XGBClassifier

clf = XGBClassifier()

In [ ]:
from sklearn.linear_model import LogisticRegressionCV

clf = LogisticRegressionCV(Cs=10, solver='saga', cv=3, max_iter=1000, n_jobs=8, verbose=0, refit=True, intercept_scaling=1.0)

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Set the parameters by cross-validation
# tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
#                      'C': [1, 10, 100, 1000]},
#                     {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

tuned_parameters = [{'n_estimators': [10, 50, 100], 'max_features': [11, 40, 121]}]

scores = ['precision']

X_train, X_test, y_train, y_test = train_test_split(inputs, outputs, test_size=0.5, random_state=0)


clf = GridSearchCV(RandomForestClassifier(), tuned_parameters, cv=5,
                       scoring='%s_macro' % 'precision')
clf.fit(X_train, y_train)

print("Best parameters set found on development set:")
print()
print(clf.best_params_)
print()
print("Grid scores on development set:")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
print()

print("Detailed classification report:")
print()
print("The model is trained on the full development set.")
print("The scores are computed on the full evaluation set.")
print()
y_true, y_pred = y_test, clf.predict(X_test)
print(classification_report(y_true, y_pred))
print()


In [12]:
# from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

# print(inputs)
# print(outputs)
# limit = len(inputs)
# scores = cross_val_score(clf, inputs, outputs, cv=5)

train_size = int(len(inputs) * 0.9)
test_size = len(inputs) - train_size



clf = clf.fit(inputs[:train_size], outputs[:train_size])
preds = clf.predict(inputs[train_size:])
correct = outputs[train_size:]

# print(scores)

print(accuracy_score(correct, preds))

0.6045691248110197


In [ ]:
import pandas as pd
import numpy as np

heroes = pd.read_csv('dota-2-match-prediction-master/data/MatchOverviewTraining.csv').values
df_extra = pd.read_csv('dota-2-match-prediction-master/data/MatchDetail.csv')

In [ ]:
print(heroes)

In [ ]:
def gen_data(heroes, df_extra):
    matches = np.zeros((10000, 121), dtype=int)
    out = np.zeros((10000), dtype=int)
    
    for (i, line) in enumerate(heroes):
        out[i] = 1 if line[-1] else 0
        match_id = line[0]
        heroes = line[1:]
        rad = heroes[:5]
        dire = heroes[5:-1]
        
        for hero in rad:
            matches[i][hero] = 1
            
        for hero in dire:
            matches[i][hero] = -1
            
        
#         counter = 121
        
#         exp_gold = df_extra.loc[df_extra['match_id'] == match_id].values
#         for (j, data) in enumerate(exp_gold):
#             matches[i][counter + 2*j] = data[4]
#             matches[i][counter + 2*j + 1] = data[14]
            
    return matches, out
        
        
        

In [ ]:
input, output = gen_data(heroes, df_extra)

In [ ]:
print(input[0])

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD, Adagrad, Adam
from keras import regularizers

model = Sequential()

train_size = int(len(input) * 0.9)
test_size = len(input) - train_size
train_in = input[ : train_size]
train_out = output[: train_size]
test_in = input[train_size : ]
test_out = output[train_size : ]

model.add(Dense(100, activation='relu', input_dim=len(input[0])))
#model.add(Dropout(0.1))
# model.add(Dense(100, activation='relu'))
# model.add(Dropout(0.5))
model.add(Dense(10, activation='relu'))
#model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))

#sgd = SGD(lr=0.05)
model.compile(loss='binary_crossentropy',
              #optimizer=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              optimizer='adadelta',
              #optimizer=sgd,
              metrics=['binary_accuracy'])

model.fit(train_in, train_out,
          epochs=200,
          validation_data=(test_in, test_out))

result = model.evaluate(test_in, test_out)

print(result[1])

In [29]:
from xgboost import XGBClassifier

clf = XGBClassifier()

In [ ]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

In [35]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()

In [ ]:
#print(input[0])
clf.predict_proba([input[1]])

In [37]:
#from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

# print(inputs)
# print(outputs)
#limit = len(inputs)
#scores = cross_val_score(clf, inputs[:limit], outputs[:limit], cv=2)

factor = 0.9
train_size = int(len(inputs) * factor)
test_size = len(inputs) - train_size
train_in, train_out, test_in, test_out = inputs[:train_size], outputs[:train_size], inputs[train_size:], outputs[train_size:]

# print(train_out, test_out)

clf = clf.fit(train_in, train_out)
preds = clf.predict(test_in)
correct = test_out

# print(scores)

print(accuracy_score(correct, preds))

0.6045691248110197


In [26]:
match = inputs[0]
print_heroes(match)
print(match)

Rad: ['crystal_maiden', 'vengefulspirit', 'windrunner', 'bristleback']
Dire: ['antimage', 'bloodseeker', 'zuus', 'lion', 'riki']
[ 0. -1.  0.  0. -1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  1.  1. -1.  0.  0.  0. -1.  0.  0.  0.  0.  0. -1.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


In [25]:
match[114] = 0

In [15]:
match[22] = 0
print_heroes(match)
print(match)

Rad: ['crystal_maiden', 'vengefulspirit', 'windrunner', 'bristleback', 'monkey_king']
Dire: ['bloodseeker', 'lion', 'riki']
[ 0.  0.  0.  0. -1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  1.  1.  0.  0.  0.  0. -1.  0.  0.  0.  0.  0. -1.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]


In [40]:
best = 0
best_chance = 0

for i in range(120):
    if match[i] != 0: continue
    match[i] = 1
    chance_win = clf.predict_proba([match])[0][1]
    
    if chance_win > best_chance:
        best_chance = chance_win
        best = i
        
    print('chance: %f hero: %s' % (chance_win, idToHero.get(i, "N/A")))
    match[i] = 0

print('BEST:', idToHero[best])
match[0] = 0
print_heroes(match)

# print(len([v for v in match if v != 0]))


chance: 0.437035 hero: axe
chance: 0.318049 hero: bane
chance: 0.404187 hero: drow_ranger
chance: 0.376936 hero: earthshaker
chance: 0.381940 hero: juggernaut
chance: 0.385745 hero: mirana
chance: 0.333265 hero: morphling
chance: 0.310563 hero: nevermore
chance: 0.403231 hero: phantom_lancer
chance: 0.243094 hero: puck
chance: 0.401993 hero: pudge
chance: 0.317593 hero: razor
chance: 0.326704 hero: sand_king
chance: 0.347727 hero: storm_spirit
chance: 0.371615 hero: sven
chance: 0.254329 hero: tiny
chance: 0.341037 hero: kunkka
chance: 0.358562 hero: N/A
chance: 0.298482 hero: lina
chance: 0.380851 hero: shadow_shaman
chance: 0.349853 hero: slardar
chance: 0.369943 hero: tidehunter
chance: 0.359329 hero: witch_doctor
chance: 0.386526 hero: lich
chance: 0.358071 hero: enigma
chance: 0.330757 hero: tinker
chance: 0.352275 hero: sniper
chance: 0.392958 hero: necrolyte
chance: 0.407136 hero: warlock
chance: 0.355123 hero: beastmaster
chance: 0.316088 hero: queenofpain
chance: 0.353165 hero